In [21]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta


/tmp/ipykernel_13803/3221308942.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [16]:
from lib.pg_connect import PgConnect, ConnectionBuilder

In [27]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta

from lib.pg_connect import PgConnect, ConnectionBuilder
# Define the default arguments for the DAG
default_args = {
    'owner': 'your_name',
    'start_date': datetime(2023, 11, 3),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
dag = DAG(
    'fetch_data_from_postgresql',
    default_args=default_args,
    schedule_interval='@daily',  # Run once daily
    catchup=False,
    tags=['data_fetch'],
)

# Define a Python function to fetch data from PostgreSQL
def fetch_data_from_postgresql():
    print('trying')
    # Create a connection to the PostgreSQL database
    pg_source = ConnectionBuilder.pg_conn('PG_ORIGIN_BONUS_SYSTEM_CONNECTION')

    # Define your SQL query to fetch data
    sql_query = """
    SELECT *
    FROM your_table;
    """

    # Execute the SQL query using the context manager
    with pg_source.connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql_query)
            result = cur.fetchall()
            # You can process the 'result' here, or simply fetch it and not send it anywhere
    print(result)

# Create a PythonOperator to run the function
fetch_data_task = PythonOperator(
    task_id='fetch_data_from_postgresql_task',
    python_callable=fetch_data_from_postgresql,
    dag=dag,
)



/tmp/ipykernel_13803/1338772803.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

/tmp/ipykernel_13803/1338772803.py:15 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [28]:
fetch_data_from_postgresql()

trying
[2023-11-03T14:24:52.000+1100] {connection.py:471} ERROR - Unable to retrieve connection from secrets backend (MetastoreBackend). Checking subsequent secrets backend.
Traceback (most recent call last):
  File "/home/vp/projects/yandex-de/sprint5/s5-lessons/venv/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1910, in _execute_context
    self.dialect.do_execute(
  File "/home/vp/projects/yandex-de/sprint5/s5-lessons/venv/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 736, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: connection

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/vp/projects/yandex-de/sprint5/s5-lessons/venv/lib/python3.10/site-packages/airflow/models/connection.py", line 466, in get_connection_from_secrets
    conn = secrets_backend.get_connection(conn_id=conn_id)
  File "/home/vp/projects/yandex-de/sprint5/s5-les

AirflowNotFoundException: The conn_id `PG_ORIGIN_BONUS_SYSTEM_CONNECTION` isn't defined

In [ ]:

# Set the task dependencies, add other tasks as needed
# Add other tasks as needed

fetch_data_task

if __name__ == "__main__":
    dag.cli()

In [33]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.hooks.postgres_hook import PostgresHook
from datetime import datetime

# Define your custom function to retrieve columns from tables
def get_columns_from_postgres_table(table_name, connection_id, **kwargs):
    # Create a PostgresHook to connect to the database
    postgres_hook = PostgresHook(postgres_conn_id=connection_id)
    
    # Fetch the columns from the specified table
    columns = postgres_hook.get_columns(table_name)
    
    # Print the columns for the table
    print(f"Columns in table '{table_name}':")
    for column in columns:
        print(f"Name: {column[0]}, Type: {column[1]}, Nullable: {column[6]}")

# Define your Airflow DAG
dag = DAG('fetch_postgres_columns', start_date=datetime(2023, 11, 3), schedule_interval=None)

# Define the tasks to fetch columns for 'ranks' and 'users' tables
get_ranks_columns = PythonOperator(
    task_id='get_ranks_columns',
    python_callable=get_columns_from_postgres_table,
    op_args=['ranks', 'PG_ORIGIN_BONUS_SYSTEM_CONNECTION'],
    provide_context=True,
    dag=dag,
)

get_users_columns = PythonOperator(
    task_id='get_users_columns',
    python_callable=get_columns_from_postgres_table,
    op_args=['users', 'PG_ORIGIN_BONUS_SYSTEM_CONNECTION'],
    provide_context=True,
    dag=dag,
)

# Set the task dependencies
get_ranks_columns >> get_users_columns


/tmp/ipykernel_13803/242399977.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

/tmp/ipykernel_13803/242399977.py:3 DeprecationWarning: The `airflow.hooks.postgres_hook.PostgresHook` class is deprecated. Please use `'airflow.providers.postgres.hooks.postgres.PostgresHook'`.

ImportError: Could not import `airflow.providers.postgres.hooks.postgres.PostgresHook` while trying to import `airflow.hooks.postgres_hook.PostgresHook`.